In [1]:
import os, sys
import numpy as np
sys.path.append('./../')
import snn

In [2]:
# target
t_rate = 'rate'
t_phase = 'phase'

# setup paths
pwd = '/users/fabianschneider/desktop/university/master/BIC/project/sim_wordlearning/'
bwd_rate = os.path.join(pwd, t_rate, 'behaviour')
mwd_rate = os.path.join(pwd, t_rate, 'models')
bwd_phase = os.path.join(pwd, t_phase, 'behaviour')
mwd_phase = os.path.join(pwd, t_phase, 'models')

# setup parameters
EPOCHS = 5

In [3]:
# setup functions
def find_files(f, t, o = None):
    '''
    Grab all files of type t from f (if o != None, selects by beginning)
    '''

    af = os.listdir(f)
    at = []
    for f in af:
        if f.endswith(t) and (o is None or f.startswith(o)):
            at.append(f)
    return at

def run_classifier(t, pwd, bwd, mwd, EPOCHS):
    # grab model to identify neurons
    with snn.Network(build_from = os.path.join(mwd, '{:s}_clean_p0.npy'.format(t))) as net:
        pop_ex = net.structure(n = 50)
        pop_ro = net.structure(n = 60)
        pop_fb = net.structure(n = 60)
    neurons = net.neurons_in(pop_ro)
    
    # iterate over epochs
    for EPOCH in np.arange(0, EPOCHS, 1):
        # find epoch
        this_epoch = find_files(bwd, '.npy', o = 'rate_clean_e{:d}'.format(EPOCH))

        # setup counts
        Y = np.zeros((60, len(this_epoch)))
        N = np.zeros((60, len(this_epoch)))

        # iterate over episodes to find spike counts per neuron per episode
        for i, episode in zip(np.arange(0, len(this_epoch), 1), this_epoch):
            # identify item
            S, I, V = (int(episode.split('.')[0].split('_')[-3]), int(episode.split('.')[0].split('_')[-2]), int(episode.split('.')[0].split('_')[-1]))

            # load behaviour
            with open(os.path.join(bwd, episode), 'rb') as f:
                M = np.load(f)
                M = M.T # note: monitors are saved as time x neuron, so transpose

            # count spikes
            N[:, i] = np.sum(M[neurons,:], axis = 1)
            Y[:, i] = I

        # setup labels
        labels = np.zeros((N.shape[0],1))

        # find maximum responses and label neurons
        max_indices = np.argmax(N, axis = 1)
        max_full = np.unravel_index(max_indices, N.shape)
        labels = Y[max_full]

        # setup matrix for Expectation-Observation matching
        T = np.zeros((len(this_epoch), 2)).astype(np.int)

        for i, episode in zip(np.arange(0, len(this_epoch), 1), this_epoch):
            # identify item
            S, I, V = (int(episode.split('.')[0].split('_')[-3]), int(episode.split('.')[0].split('_')[-2]), int(episode.split('.')[0].split('_')[-1]))

            # load behaviour
            with open(os.path.join(bwd, episode), 'rb') as f:
                M = np.load(f)
                M = M.T # note: monitors are saved as time x neuron, so transpose

            # label current expectation
            T[i, 0] = int(I)

            # label current observation
            T[i, 1] = int(labels[np.argmax(np.sum(M[neurons,:], axis = 1))])

        # get accuracy
        A = np.mean((T[:,0] == T[:,1]).astype(np.int))
        print('Epoch{:d}, accuracy = {:3.4f}'.format(EPOCH, A))

In [4]:
run_classifier(t_rate, pwd, bwd_rate, mwd_rate, EPOCHS)

Epoch0, accuracy = 0.024
Epoch1, accuracy = 0.022
Epoch2, accuracy = 0.024
Epoch3, accuracy = 0.024
Epoch4, accuracy = 0.021


In [80]:
run_classifier(t_phase, pwd, bwd_phase, mwd_phase, EPOCHS)

Epoch0, accuracy = 0.024
Epoch1, accuracy = 0.022
Epoch2, accuracy = 0.024
Epoch3, accuracy = 0.024
Epoch4, accuracy = 0.021
